# Inferencia con LLamaCpp

## Instalación de paquetes de python

In [ ]:
!pip install  prompttools typing sentence-transformers

# Instalación de llama-cpp-python

In [4]:
!CMAKE_ARGS="-DLLAMA_AVX=on -DLLAMA_AVX2=on -DLLAMA_F16C=on -DLLAMA_FMA=on" .venv/bin/pip install llama-cpp-python --upgrade --force-reinstall --no-cache-dir

#!CMAKE_ARGS="-DLLAMA_CUDA=on" .venv/bin/pip install llama-cpp-python --force-reinstall --no-cache-dir --upgrade --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu123

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 MB 115.4 MB/s eta 0:00:001m115.7 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 357.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 361.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 456.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 115.4 MB/s eta 0:00:00 MB/s eta 0:00:0101
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.78-cp311-cp311-linux_x86_64.whl size=3749804 sha256=b4aa1e01f73f232c3f7742d66861428671f624218dd51af4d0bf2108686a2e7c
  Stored in directory: /run/user/1000/pip-ephem-wheel-cache-dx0_er86/wheels/90/04/d0/e152de3e3f4ecf76f4a0c379e2806cc8a9700c0384b82cbd2f
Successfully built llama-cpp-python
  Attempting uninst

## Importamos `prompttools` y los modulos de **typings**

In [ ]:
from typing import Dict, List, Tuple
from prompttools.experiment import LlamaCppExperiment

# Descargamos los modelos cuantizados para usar con llama-cpp

[Referencia de tipos de cuantización empleados en llama.cpp y cómo distinguirlos](https://github.com/ggerganov/llama.cpp/wiki/Tensor-Encoding-Schemes)

In [ ]:
!mkdir -p models
!curl -sSLk https://huggingface.co/TheBloke/LLaMa-7B-GGML/blob/main/llama-7b.ggmlv3.q2_K.bin -O --output-dir models
!curl -sSLk https://huggingface.co/TheBloke/Llama-2-7B-GGML/blob/main/llama-2-7b.ggmlv3.q2_K.bin -O --output-dir models

> OJO! ¿Lo hemos descargado bien? ;-) Miremos...

In [ ]:
!ls -lh models/llama*
!file models/llama-2-7b.ggmlv3.q2_K.bin
!file models/llama-2-7b.ggmlv3.q2_K.bin


Cuidado con las descargas directas de HF...

In [ ]:
!pwd

En la carpeta **models**/buenos/** , hemos descargado directamente los modelos adecuados.

- [TheBloke - LLama-7b GGUF](https://huggingface.co/TheBloke/LLaMA-7b-GGUF/tree/main)
- [TheBloke - LLama-2-7b GGUF](https://huggingface.co/TheBloke/Llama-2-7B-GGUF)


Si quieres saber que es el formato GGUF, que se emplea con llama.cpp, mira [este artículo](https://vickiboykis.com/2024/02/28/gguf-the-long-way-around/).

Y ojo con los modelos que cargas. Mira este [vídeo explicativo](https://youtu.be/2ethDz9KnLk) sobre el tema.

Ahora crearemos nuestros prompts a comparar así como la configuración de los modelos que queremos comparar y el parámetro de la temperatura.

[LlamaCppExperiment](https://github.com/hegelai/prompttools/blob/main/prompttools/experiment/experiments/llama_cpp_experiment.py)

In [ ]:
model_paths = [
    "/home/guachu/Desktop/practica/inferencia_con_llamacpp/models/buenos/llama-7b.Q2_K.gguf",  
    "/home/guachu/Desktop/practica/inferencia_con_llamacpp/models/buenos/llama-2-7b.Q2_K.gguf",
]  
prompts = [
    "Who was the first president?",
    "Who was the first president of the USA?",
]
temperatures = [0.0, 1.0]
max_tokens = [20]


call_params = dict(temperature=temperatures,max_tokens=max_tokens)

experiment = LlamaCppExperiment(model_paths, prompts, call_params=call_params)

Ejecutamos el experimento una vez que lo hemos configurado

In [ ]:
experiment.run()

## Evaluación de los modelos

Definimos una función de evaluación. En este caso, la de similaridad semántica


In [ ]:
from prompttools.utils import semantic_similarity

In [ ]:
experiment.evaluate("similar_to_expected", semantic_similarity, expected=["George Washington"] * 8)

Finalmente, visualizamos el resultado del experimento


In [ ]:
experiment.visualize()